<center>
    <b> Microproyecto #7 </b>
    <br/> Javier Anleu 17149
    <br/> Andrea Argüello 17801
</center>

# Función inversa de distribución exponencial, E[X] y Var(X)

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import display, Latex

In [2]:
exponentialInv = lambda x,lam: -np.log(1-x)/lam # donde x es U(0,1)
mu_exponencial = lambda lam: 1/lam
sigma_cuadrado_exponencial = lambda lam: 1/(lam**2)

## Ejercicio 1: Variables antitéticas, $\lambda = 1$ 

In [3]:
lambda_1 = 1

In [4]:
random.seed(500)
def ej1(repeticiones):
    X1 = []
    X2 = []
    Y = []
    for i in range(repeticiones):
        x = 1
        while x == 1:
            x = random.uniform(0,1)
        X1.append(exponentialInv(x,lambda_1))
        X2.append(exponentialInv(1-x,lambda_1))
        Y.append((X1[-1]+X2[-1])/2)
    return X1, X2, Y

In [5]:
cov_xy = lambda X, Y: np.mean([x * y for x, y in zip(X, Y)])-np.mean(X)*np.mean(Y)

In [6]:
var = lambda X: np.mean([x**2 for x in X])-(np.mean(X))**2

In [7]:
var_mean_xy = lambda X, Y: (var(X) + var(Y) + 2*cov_xy(X,Y))/4

In [8]:
X1, X2, Y = ej1(1000)

In [9]:
display(Latex(f'$Cov(X_1,X_2)$ = %f' % (cov_xy(X1,X2))))

<IPython.core.display.Latex object>

Queda demostrado que la covarianza entre $X_1$ y $X_2$ es negativa. Como ambas provienen de la distribución exponencial, entonces $X_1$ y $X_2$ son antitéticas.

In [10]:
display(Latex(f'$E[Y]$ obtenido: %f, $\mu$ esperado: %f' % (np.mean(Y), mu_exponencial(lambda_1))))

<IPython.core.display.Latex object>

In [11]:
display(Latex(f'$Var(Y)$ obtenida: %f, $\sigma^2$ esperada: %f' % (var(Y), sigma_cuadrado_exponencial(lambda_1))))

<IPython.core.display.Latex object>

## Ejercicio 2: Variables de control, $\lambda = 1$ 

In [21]:
random.seed(487)
def ej2(repeticiones):
    X = []
    Y = []
    for i in range(repeticiones):
        x = 1
        y = 1
        while x == 1:
            x = random.uniform(0,1)
        while y == 1:
            y = random.uniform(0,1) # sabemos que mu_y = 0.5 y var_y = 1/12
        X.append(exponentialInv(x,lambda_1))
        Y.append(y)
    return X, Y

def c_asterisco(lista1,lista2): #Basado en el ejemplo visto en clase
    n=0
    d=0
    promedio_x=np.mean(lista1)
    promedio_y=np.mean(lista2)
    for i in range(len(lista1)):
        n=n+(lista1[i]-promedio_x)*(lista2[i]-promedio_y)
        d=d+pow(lista2[i]-promedio_y,2)
    return -1*n/d
        

control = lambda X, Y, mu_y, c: [x + c*(y - mu_y) for x, y in zip(X,Y)]

X, Y = ej2(1000)

#A esta covarianza le pasamos el mu teórico de la distribución Y
cov_xy_muy = lambda X, Y, mu_y: np.mean([x * y for x, y in zip(X, Y)])-(np.mean(X)*mu_y) 

c_min = c_asterisco(X, Y)

In [22]:
display(Latex(f'$E[X]$ obtenido: %f, $\mu$ esperado: %f' % (np.mean(control(X, Y, 0.5, c_min)), mu_exponencial(lambda_1))))

<IPython.core.display.Latex object>

In [23]:
display(Latex(f'$Var(X)$ obtenida: %f, $\sigma^2$ esperada: %f' % (var(control(X, Y, 0.5, c_min)), sigma_cuadrado_exponencial(lambda_1))))

<IPython.core.display.Latex object>